This notebook was made after title.ipynb.

Please refer to the title.ipynb for further explanation

Now loading the dataset which contains only questions(from title.ipynb)


In [1]:
import json
import requests
import time
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np

In [2]:
def print_full(x):
    pd.set_option('display.max_rows', len(x))
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', 2000)
    pd.set_option('display.float_format', '{:20,.2f}'.format)
    pd.set_option('display.max_colwidth', None)
    print(x)
    pd.reset_option('display.max_rows')
    pd.reset_option('display.max_columns')
    pd.reset_option('display.width')
    pd.reset_option('display.float_format')
    pd.reset_option('display.max_colwidth')

In [3]:
with open('/content/questions_related.json') as j :
  questions = pd.read_json(j,orient='table') ### Remember the orientation is table and index is id
print_full(questions.head(5))
questions.info()

         AcceptedAnswerId          AnswerCount                                                                                                                                 AnswerIds                                             Attachments       OS  PostTypeId                                                          Related  Score                                              Tags  \
Id                                                                                                                                                                                                                                                                                                                                                                                                
1              727,273.00                12.00                                            [8517, 10937, 13975, 14035, 255395, 372191, 594216, 656257, 727273, 1164321, 1183269, 1326351]                        [images/1_1.png, i

In [4]:
#removing queries with no title at all
questions.dropna(axis=0,subset=['Title'],inplace=True)
questions.isnull().sum()
# print(questions.tail())

AcceptedAnswerId    30640
AnswerCount             0
AnswerIds               0
Attachments         45658
OS                  45658
PostTypeId              0
Related             15283
Score                   0
Tags                    0
Text                    0
Title                   0
dtype: int64

A sample use of stackexchange api for retreiving of related field

In [12]:
tokens = {1:{'access_token': 'grmO1cXoVtH4cQKNGKQoQw))','key':'NgwXtZS5zddMDTN5ch8kUQ(('},
          2:{'access_token': 'pe*iEaXctSGrih1mUvrIbA))','key':'t5sJBRionMBtc05aetiCmw(('},
          3:{'access_token': '4y4d4geC36lSDcJTpqx1aQ))','key':'j3YCFivyCrWDhDH*ltdokQ(('},
          4:{'access_token': 'eJS8fzitX3qDl2AnJY*P8g))','key': 'Cp1AgenjkXQ03I7avs*jDg(('},
          5:{'access_token': 'hEP9Sl6CZDgx9RzSx9BLmw))','key':'b6*O3b8)i5eVM4CdcoQ6IA(('}
          }
print(tokens[2]['access_token'])
token_counter = 3

pe*iEaXctSGrih1mUvrIbA))


In [13]:
def limit_checker(i):
    url = f"https://api.stackexchange.com/2.2/questions/1/related?access_token={tokens[i]['access_token']}&key={tokens[i]['key']}&order=desc&sort=activity&site=superuser&filter=!--Fgz)TFKJfq"
    a = requests.get(url)
    print(a)
    related = json.loads(a.content)
    
    # print(related)
    return related['quota_remaining']


In [14]:
limit = limit_checker(token_counter)
print(limit)

<Response [200]>
9999


In [15]:
cnt = 0 
for i in questions.index.values:
    if(cnt>limit):
        if(token_counter==3):
            print('All the tokens has been exhausted')
            break
        print('Token no- '+str(token_counter)+'has been exhausted. No of queries served = '+str(cnt))
        token_counter+=1
        cnt=0
        limit = limit_checker(token_counter)
    if(questions.loc[i,'Related']==None):
        start = time.time()
        url = f"https://api.stackexchange.com/2.2/questions/{i}/related?access_token={tokens[token_counter]['access_token']}&key={tokens[token_counter]['key']}&order=desc&sort=activity&site=superuser&filter=!--Fgz)TFKJfq"
        a = requests.get(url)
        related = json.loads(a.content)
        try:
            questions.at[i,'Related'] = []
            for j in related['items']:
                questions.at[i,'Related'].append(j['question_id'])
            cnt+=1
            if(cnt%100==0):
                print(cnt,related['quota_remaining'])
        except Exception as e:
            print('Related field not found',e)
            cnt+=1
        end = time.time()
        if(end-start<0.04):
            time.sleep(0.04-(end-start))
        

100 9900
200 9800
300 9700
400 9599
500 9499
600 9400


In [16]:
questions.to_json('/content/questions_related.json',orient='table') 
# Uncomment only when u are sure to change the file. Once the change has been made. it is not possible to revert

In [17]:
print(questions.isnull().sum())

AcceptedAnswerId    30640
AnswerCount             0
AnswerIds               0
Attachments         45658
OS                  45658
PostTypeId              0
Related                 0
Score                   0
Tags                    0
Text                    0
Title                   0
dtype: int64


In [18]:
print_full(questions.tail(5))

            AcceptedAnswerId          AnswerCount AnswerIds Attachments    OS  PostTypeId                                                                                 Related  Score                                                          Tags  \
Id                                                                                                                                                                                                                                                       
1474210                  nan                 1.00        []        None  None           1        [1352676, 841632, 785554, 749407, 88073, 100730, 634882, 585947, 456590, 334811]      0  [networking, wireless-networking, proxy, windows-networking]   
1474386                  nan                 1.00        []        None  None           1  [1219223, 1431820, 587067, 993986, 1032894, 1105070, 1086312, 1053978, 750603, 366140]      0                                          [networking, router]   


In [19]:
for i in questions.index.values:
    if(len(questions.at[i,'Related']) == 0):
        questions.at[i,'Related'] = None

In [20]:
alpha =0
for i in questions.index.values:
    if((questions.at[i,'Related']) == None and questions.at[i,'Attachments']!=None):
        alpha+=1
print(questions.isnull().sum())
print(alpha)

AcceptedAnswerId    30640
AnswerCount             0
AnswerIds               0
Attachments         45658
OS                  45658
PostTypeId              0
Related               316
Score                   0
Tags                    0
Text                    0
Title                   0
dtype: int64
2


ON Analysis i found out there are 97 Non-attachmnet queries with no 'related' field and 15 Attachment queries with no 'related' field